In [2]:
from utils import read_bills
import regex as re
import os
from typing import Dict

# Data loading

In [5]:
current_dir = os.getcwd()
bills_dict = read_bills(f"{current_dir}/data/first_ex_data/ustawy")

# 1